# PAMM Clustering - Whole Dataset

Sample Notebook to use PAMM clustering algorithm (orignal [paper](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.7b00993)) with the GMPLabTools implementation.

The keyword **WHOLE** dataset refers to the tratments of the dataset towards the kernel density estimation (KDE), which are "summed" togheter.

In [1]:
import time
import warnings
import random
import seaborn as sns
import numpy as np

from scipy.cluster.hierarchy import dendrogram

from gmplabtools.analysis import DataSampler
from gmplabtools.pamm import PammGMM
from gmplabtools.pamm import Pamm
from gmplabtools.analysis import calculate_adjacency, adjancency_dendrogram
from gmplabtools.analysis import ClusterRates

import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'gmplabtools'

## Utilities Functions

In [ ]:
def make_colors(clust,mode='tab20'):
    if np.min(clust) == -1:
        N = np.unique(clust).shape[0] - 1
        colors = sns.color_palette(mode, N) + [(0,0,0)]
    else:
        N = np.unique(clust).max()
        colors = sns.color_palette(mode, N) 
    return colors


def get_axes(L, max_col=3, fig_frame=(5,4), res=100):
    cols = L if L <= max_col else max_col
    rows = int(L / max_col) + int(L % max_col != 0)
    fig, ax = plt.subplots(rows, cols, figsize=(cols * fig_frame[0], rows * fig_frame[1]), dpi=res)
    ax =  ax.flatten()
    return fig, ax


def shuffle(X, Y=None, n=None):
    l = np.arange(X.shape[0])
    random.shuffle(l)
    if Y is None:
        return X[l[:n],:]
    elif Y is None and n is None:
        return X[l,:]
    elif n is None:
        return X[l,:], Y[l]
    else: 
        return X[l[:n],:], Y[l[:n]]



## Dataset definition and loading

The data that one wants to process needs to be load and initialized as follows

`SYSX1 = np.loadtxt(my_dir/my_fileX1)`

and then put in a well named dictionary

`SYST = {
    'name_X1' : SYSX1,
    'name_X2' : SYSX2,
        ...   : ...  ,
}`

As stated before in this workflow one need to define a _wholesystemData_ and store it accordingly

`ALL = np.loadtxt(my_dir/my_wholedata)`

In [ ]:
PCA_DIR='/home/andreag/SOFTSYSTEMS/FIBERSminimal_10mu/pca_files/rcut8'

In [ ]:
SYS1 = np.loadtxt(PCA_DIR+'/PCA_fiber1_c1_10mu_ev10ns_rcut8_trj0-1001-1.pca')
SYS2 = np.loadtxt(PCA_DIR+'/PCA_fiber2_c5_10mu_ev10ns_rcut8_trj0-1001-1.pca')
SYS3 = np.loadtxt(PCA_DIR+'/PCA_fiber3_n0_10mu_ev10ns_rcut8_trj0-1001-1.pca')
ALL = np.loadtxt(PCA_DIR+'/wholesystem.pca')

In [ ]:
SYST = {
    'fib1' : SYS1,
    'fib2' : SYS2,
    'fib3' : SYS3,
}

In [ ]:
DIM = ALL.shape[1]
print(f"Data dimensions considered: {DIM}")

## General variables

In [ ]:
CHUNK=5000
LABEL_SIZE=18
L=len(SYST)

In [ ]:
shffull = shuffle(ALL)

fig, ax = plt.subplots()
ax.scatter(shffull[:CHUNK,0], shffull[:CHUNK,1])
ax.set_title('whole data visualization')
gx = ax.get_xlim()
gy = ax.get_ylim()

## Algorithm inputs

The paramters for the calculation needs to be stored as follows.

The meaning of these parameters can be found in the orignal [paper](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.7b00993).

The `nm_frame` refers to how many components a frame of the trajectory is composed (es. fiber having 40 monomers `nm_frame : 40`).

In [ ]:
default_inputs = dict(
    # cluster
    distance = "minkowski",
    size = 2000,
    p = 2,
    generate_grid = True,
    savegrid = "grid_data",
    # cluster inputs
    d = DIM,
    fspread = 0.25,
    ngrid = 2000,
    qs = 1,
    o = "pamm",
    trajectory = PCA_DIR+"/wholesystem.pca",
    readgrid = "grid_data",
    merger = 0.01,
    bootstrap = 73
)

In [ ]:
datasets_cluster = [
    (ALL, {}),
]

datasets_predict = [
    (SYS1, {'sys' : 'fib1', 'nm_frame' : 40}),
    (SYS2, {'sys' : 'fib2', 'nm_frame' : 40}),
    (SYS3, {'sys' : 'fib3', 'nm_frame' : 40})
]

## Original dataset plot

In [ ]:
colors=sns.color_palette('tab10', L)
fig, ax = get_axes(L, max_col=L)
for i,s in enumerate(SYST):
    ax[i].scatter(SYST[s][:CHUNK,0], SYST[s][:CHUNK,1], s=10, linewidth=1, marker="o", alpha=0.5)
    ax[i].set_xlim(gx)
    ax[i].set_ylim(gy)
    ax[i].set_title(f"{s}", weight='bold',size=LABEL_SIZE)
    ax[i].tick_params(labelsize=LABEL_SIZE,width=3,size=7)
    
    for side in ['bottom','right','top','left']:
        ax[i].spines[side].set_linewidth(3)
    
    if i == 0:
        ax[i].set_xlabel('PCA 1', weight='bold',size=LABEL_SIZE)
        ax[i].set_ylabel('PCA 2', weight='bold',size=LABEL_SIZE)
        for side in ['right','top']:
            ax[i].spines[side].set_visible(False)          
    else:
        ax[i].set_xlabel('PCA 1', weight='bold',size=LABEL_SIZE)
        ax[i].tick_params(labelleft=None)
        for side in ['right','top']:
            ax[i].spines[side].set_visible(False)

fig.tight_layout()
# plt.savefig("data_set_soap_pca.png")

## PAMM - Clustering part

In [ ]:
for i_dataset, (dataset, algo_params) in enumerate(datasets_cluster):
    # update parameters with dataset-specific values
    params = default_inputs.copy()
    params.update(algo_params)

    # Clustering
    p = Pamm(params)
    print('\n#-----------------------------------------------')
    print(p.command_parser)
    
    print('\nRUNNING Clustering')
    t0 = time.time()
    p.run()
    t1 = time.time()
    print('TIME= '+str(np.round(t1-t0, 2))+' s \n')

## PAMM - Prediction on data part

In [ ]:
gmm = PammGMM.read_clusters('pamm.pamm', 
                                grid_file='pamm.grid', 
                                bootstrap_file='pamm.bs')
NUM_CLUST=np.unique(gmm.pk).shape[0]
print(f"There are {NUM_CLUST} clusters")

In [ ]:
cluster_output = {}
grid_cluster = {}
prob_output = {}
bootstr_output = {}
systnames = []
for i_dataset,dataset in enumerate(datasets_predict):
    run_syst = str(datasets_predict[i_dataset][1]['sys'])
    # Predict
    print('\nRUNNING Predict '+run_syst)
    t0 = time.time()
    
    x = datasets_predict[i_dataset][0]
    x_ = gmm.predict_proba(x)
    labels = np.argmax(x_, axis=1) #.reshape((-1, 1))

    t1 = time.time()
    print('TIME= '+str(np.round(t1-t0, 2))+' s \n')

    # Storing data
    cluster_output[run_syst] = labels
    grid_cluster[run_syst] = gmm.cluster
    prob_output[run_syst] = gmm.p
    bootstr_output[run_syst] = gmm.bs
    systnames.append(run_syst)

    # output for initial clustering
    np.savetxt(run_syst + "_clusters.txt", labels.reshape((-1, 1)))
    
    rates = ClusterRates(datasets_predict[i_dataset][1]['nm_frame'], 'label').calculate_matrix(labels.reshape((-1, 1)))
    np.savetxt(run_syst + "_rates.txt", rates)

## Output post-processing

In [ ]:
def PLTmatrixrates(ax,data,s=18):
    labels = ['0','1', '2', '3', '4', '5', '6','7','8']
    sns.heatmap(data, annot=True, fmt=".2f", cbar=False, ax=ax, annot_kws={"fontsize":s})
    ax.xaxis.tick_top()
    ax.set_xticklabels(labels, size='18', weight='bold')
    ax.set_yticklabels(labels, size='18', weight='bold')
    return ax

In [ ]:
colors=make_colors(NUM_CLUST,mode='tab20')
fig, ax = get_axes(L, max_col=L)
for i,sys in enumerate(systnames):
    labels = cluster_output[sys]
    ax[i].scatter(datasets_predict[i][0][:CHUNK,0], datasets_predict[i][0][:CHUNK,1], c=np.array(colors)[labels[:CHUNK]], s=10)
    ax[i].set_xlim(gx)
    ax[i].set_ylim(gy)
    ax[i].set_title(f"{sys}", weight='bold',size=LABEL_SIZE)
    ax[i].tick_params(labelsize=LABEL_SIZE,width=3,size=7)
    
    for side in ['bottom','right','top','left']:
        ax[i].spines[side].set_linewidth(3)
    
    if i == 0:
        ax[i].set_xlabel('PCA 1', weight='bold',size=LABEL_SIZE)
        ax[i].set_ylabel('PCA 2', weight='bold',size=LABEL_SIZE)
        for side in ['right','top']:
            ax[i].spines[side].set_visible(False)          
    else:
        ax[i].set_xlabel('PCA 1', weight='bold',size=LABEL_SIZE)
        ax[i].tick_params(labelleft=None)
        for side in ['right','top']:
            ax[i].spines[side].set_visible(False)

fig.tight_layout()
# fig.savefig('clusters_pamm.png')

In [ ]:
SYS1rates = np.loadtxt("./fib1_rates.txt")
SYS2rates = np.loadtxt("./fib2_rates.txt")
SYS3rates = np.loadtxt("./fib3_rates.txt")

RATES = {
    'fib1' : SYS1rates,
    'fib2' : SYS2rates,
    'fib3' : SYS3rates,
}

In [ ]:
fig, ax = get_axes(L, max_col=3, fig_frame=(5,4), res=100)
for i,sys in enumerate(RATES):
    PLTmatrixrates(ax[i], RATES[sys], s=14)
    
fig.tight_layout()
# fig.savefig('micro_clusters_pamm_matrix.png')
#Probabilità di transizione
#Riga partenza
#Colonna arrivo

### Clusters hierarchy

In [ ]:
prob_output.keys()

In [ ]:
row = 1
col = 1
fig, ax = plt.subplots(row, col, figsize=(col * 5, row * 4), dpi=100)

adjacency, mapping = calculate_adjacency(
prob=prob_output['fib3'],
clusters=grid_cluster['fib3'],
bootstrap=bootstr_output['fib3']
)
z = adjancency_dendrogram(adjacency)
_ = dendrogram(z, ax=ax, count_sort=True)['leaves']
    
for k in range(col):
    ax.set_yticks([])
    ax.yaxis.set_ticks_position('none')
    for side in ['bottom','right','top','left']:
        ax.spines[side].set_visible(False)


# ax.set_ylabel('PAMM DENDROGRAM', size='16')
fig.savefig('clusters_pamm_dendrogram.png')

### Clusters mearging (Macroclusters processing)

Macrocluster syntax definition:

`mapping = [
    ('SYSX1', {MacroCl1: [microClx,...], 
               MacroCl2: [microCly,...]})
]`

where the mearging comes from the dendrogram.

In [ ]:
mapping = [
    ('0.12', {0: [0,1,4],
              1: [3,4]}),
    ('1.15', {0: [0,1,3,4],
             1: [2]}),
    ('2.18', {0: [0,1,3,4],
             1: [2]}),
    ('3.20', {0: [0,1,3,4],
             1: [2]}),
    ('4.24', {0: [0,1,3,4],
             1: [2]})
]

In [ ]:
datasets_predict[0][0].shape[0]

In [ ]:
# it does not matter if one put np.argmax(y__, axis=1).reshape((-1,1)) \w or \wout the reshape part
macro_cluster_output = {}
rates_macro_clusters = {}

for s,macro_cl in enumerate(systnames):
    # Macro Cluster
    run_syst = macro_cl
    print("MACRO CLUSTERS - "+run_syst)
    
    y = datasets_predict[s][0]
    y_ = gmm.predict_proba(y)
    y__ = np.zeros((y.shape[0], len(mapping[s][1])))
    for k, v in mapping[s][1].items():
        y__[:, k] = y_[:,v].sum(1)

    macro_cluster_output[macro_cl] = np.argmax(y__, axis=1)
    np.savetxt(run_syst+'_macro_cluster.dat', np.argmax(y__, axis=1).reshape((-1,1)) )
    
    rates = ClusterRates(datasets_predict[s][1]['nm_frame'], 'label').calculate_matrix(np.argmax(y__, axis=1).reshape((-1,1)) )
    rates_macro_clusters[macro_cl] = rates
    np.savetxt(run_syst+'_macro_rates.dat', rates)

In [ ]:
rates_macro_clusters

In [ ]:
Mcolors = ["tab:blue", "tab:red", "tab:green"]

fig, ax = get_axes(L, max_col=L)
for i,sys in enumerate(SYST):
    colors=Mcolors
    labels = macro_cluster_output[sys]
    ax[i].scatter(datasets_predict[i][0][:CHUNK,0], datasets_predict[i][0][:CHUNK,1], c=np.array(colors)[labels[:CHUNK]], s=10)
    ax[i].set_title(f"{sys}", weight='bold',size=LABEL_SIZE)
    ax[i].tick_params(labelsize=LABEL_SIZE,width=3,size=7)
    
    ax[i].set_xlim(gx)
    ax[i].set_ylim(gy)
    
    for side in ['bottom','right','top','left']:
        ax[i].spines[side].set_linewidth(3)
    
    if i == 0:
        ax[i].set_xlabel('PCA 1', weight='bold',size=LABEL_SIZE)
        ax[i].set_ylabel('PCA 2', weight='bold',size=LABEL_SIZE)
        for side in ['right','top']:
            ax[i].spines[side].set_visible(False)          
    else:
        ax[i].set_xlabel('PCA 1', weight='bold',size=LABEL_SIZE)
        ax[i].tick_params(labelleft=None)
        for side in ['right','top']:
            ax[i].spines[side].set_visible(False)

fig.savefig('macro_clusters_pamm.png')


In [ ]:
SYS1rates = np.loadtxt("./0.12_macro_rates.dat")
SYS2rates = np.loadtxt("./1.15_macro_rates.dat")
SYS3rates = np.loadtxt("./2.18_macro_rates.dat")
SYS4rates = np.loadtxt("./3.20_macro_rates.dat")
SYS5rates = np.loadtxt("./4.24_macro_rates.dat")
RATES = {
    '0.12' : SYS1rates,
    '1.15' : SYS2rates,
    '2.18' : SYS3rates,
    '3.20' : SYS4rates,
    '4.24' : SYS5rates,
}

fig, ax = get_axes(L, max_col=3, fig_frame=(5,4), res=100)
for i,sys in enumerate(RATES):
    PLTmatrixrates(ax[i], RATES[sys])
fig.savefig('macro_clusters_pamm_matrix.png')